In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import boto3
bucket_name='my_bucket_name'
data_set='jpeg/train'
region = boto3.Session().region_name
# s3train_path = 'https://s3-{}.amazonaws.com/{}/{}'.format(region,bucket_name, data_set)
s3train_path='s3://{}/jpeg/train/'.format(bucket_name)

In [ ]:
role= get_execution_role()
sess=sagemaker.Session()
training_image=get_image_uri(sess.boto_region_name, 'image-classification', repo_version='latest')

In [ ]:
s3_output_location='s3://{}/{}/output'.format(bucket_name, 'jpeg')

The recommended input format for the Amazon SageMaker image classification algorithms is Apache MXNet RecordIO. With that format, the training process uses the data in pipe mode rather than file mode. Pipe mode is faster because it makes an stream of data instead of copying batches of data one by one. In pipe mode the training job does not have to wait for the data to arrive before starting the process on each bach. However, we can also have jpeg or png files and make it be used in pipe by using an augmented manifest file. Find more information here: https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html

# make augmented manifest file for train and validation sets

In [ ]:
path='path_to_train.csv'
df=pd.read_csv(path)
df.loc[:, 'file_name']= df.loc[:,'image_name']+'.jpg'

In [ ]:
#make train and validation sets
from sklearn.utils import shuffle
df=shuffle(df)
val_size=int(df.shape[0]/10)
df_val=df[0:val_size]
df=df[val_size:]

In [ ]:
#duplicate mallignant cases for 55 times to make the data balanced!
df_mallignant=df[df['target']==1]
for i in range(55):
    df=df.append(df_mallignant)
df = shuffle(df)

In [ ]:
#make augmented manifest file for train dataset
df_manifest= df[['file_name', 'target']]
df_manifest['file_name']= 's3://{}/jpeg/train/'.format(bucket_name)+df_manifest['file_name']
df_manifest.columns=['source-ref', 'class']
out=json.dumps(json.loads(df_manifest.to_json(orient='records')))[1:-1].replace('}, {', "}\n{")
with open('../data/train_manifest', 'w') as f:
    f.write(out)
s3_client = boto3.client('s3')
s3_client.upload_file('../data/train_manifest', Bucket=bucket_name, Key='jpeg/train_manifest')

In [ ]:
#make augmented manifest file for validation dataset
df_val_manifest= df_val[['file_name', 'target']]
df_val_manifest['file_name']= 's3://{}/jpeg/train/'.format(bucket_name)+df_val_manifest['file_name']
df_val_manifest.columns=['source-ref', 'class']
out=json.dumps(json.loads(df_val_manifest.to_json(orient='records')))[1:-1].replace('}, {', "}\n{")
with open('../data/validation_manifest', 'w') as f:
    f.write(out)
s3_client = boto3.client('s3')
s3_client.upload_file('../data/validation_manifest', Bucket=bucket_name, Key='jpeg/validation_manifest')

# train in pipe mode with augmented manifest files

In [ ]:
# Create a model object set to using "Pipe" mode.
model = sagemaker.estimator.Estimator(training_image,
                                      role,
                                      train_instance_count=1,
                                      train_instance_type='ml.p3.2xlarge', 
                                      train_volume_size = 20, 
                                      train_max_run = 3600*5,
                                      input_mode = 'Pipe',
                                      output_path=s3_output_location,
                                      sagemaker_session=sess)
#                                     model_uri= 's3-address-of-an-existing-model')

In [ ]:
base_hyperparameters=dict(
    use_pretrained_model=1,
    image_shape='3,224,224',
    num_classes=2,
    num_training_samples=58084,
)

hyperparameters={
    **base_hyperparameters,
    **dict(
        learning_rate=0.001,
        mini_batch_size=32, 
        num_layers= 101,
        epochs=4,
        top_k=2
    )
}

model.set_hyperparameters(**hyperparameters)

In [ ]:
# Create a train data channel with S3_data_type as 'AugmentedManifestFile' and attribute names.
train_data = sagemaker.session.s3_input('s3://{}/jpeg/train_manifest'.format(bucket_name),
    distribution='FullyReplicated',
    content_type='application/x-recordio',
    s3_data_type='AugmentedManifestFile', # if recordIO files were use, it should have been set to's3Prefix'
    attribute_names=['source-ref', 'class'],
    input_mode='Pipe',
    record_wrapping='RecordIO') 

validation_data=sagemaker.session.s3_input('s3://{}/jpeg/validation_manifest'.format(bucket_name),
    distribution='FullyReplicated',
    content_type='application/x-recordio',
    s3_data_type='AugmentedManifestFile', 
    attribute_names=['source-ref', 'class'],
    input_mode='Pipe',
    record_wrapping='RecordIO') 

data_channels = {'train': train_data, 'validation':validation_data}

In [ ]:
model.fit(inputs=data_channels, job_name='my_training_job_name', logs=True)

In [ ]:
job=model.latest_training_job
job_name=job.name
print(f"available for download at: {model.output_path}/{job_name}/")

# batch inference

In [ ]:
model_addr='s3://{}/jpeg/output/my_training_job_name/output/model.tar.gz'.format(bucket_name)

In [ ]:
model = sagemaker.model.Model(
    model_data=model_addr, 
    image=training_image,
    role=role, 
    sagemaker_session=sess 
)  


In [ ]:
# The location of the test dataset
batch_input = 's3://{}/jpeg/test'.format(bucket_name)

# The location to store the results of the batch transform job
batch_output = '{}/inference_output'.format(s3_output_location)

In [ ]:
#I tried doing batch inference in pipe mode (by provding manifest files), but it does not work. 
#Does not raise an error, and does not produce the output files
#so, here I am doing it in file mode
transformer = model.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path=batch_output)
transformer.transform(data=batch_input, data_type='S3Prefix', content_type='image/jpeg', split_type=None)
transformer.wait()

In [ ]:
#this is how I tied doing the batch inference in pipe mode
test_manifest='s3://{}/jpeg/test_manifest'.format(bucket_name)
transformer= model.transformer(instance_count=1, instance_type='ml.m4.4xlarge', assemble_with='Line',
                               output_path=s3_output_location)transformer.transform(data= test_manifest , 
    data_type='ManifestFile', 
    job_name='melanoma-first-transform-job-7',#if I do not give a name, it will do so automatically with date and time
    content_type='application/x-recordio',
    split_type='Line')
transformer.wait()

# make submission file

I downloaded the whole folder that contains the output of batch transform to my own computer. Then I made the submission file as follows:

In [ ]:
import os;
import json

output = "image_name,target\n"
for file in os.listdir(path/'output_folder'):
    jpg_name = os.path.splitext(os.path.splitext(file)[0])[0]
    with open(str(path/'output_folder')+"/"+file) as f:
        data = json.load(f)
        value = round(data['prediction'][1], 4)
        output = output + jpg_name + ","+str(value) + "\n"

with open(path/'my_submission.csv', 'w') as sub:
    sub.write(output)